In [15]:
import journalism
import logging
from csv import reader

text_type = journalism.TextType()
number_type = journalism.NumberType()
date_type = journalism.DateType()

In [16]:
def get_table(datarows, types, titles):
    try:
        table = journalism.Table(datarows, types, titles)
        return table
    except:
        logging.exception('problem loading table')
    return None

In [17]:
def clean_text(row):
    new_row = []
    for item in row:
        if isinstance(item, (str, unicode)):
            item = item.decode('utf-8', 'replace')
        if item in [u'--', u'n/a']:
            item = None
        new_row.append(item)

    return new_row

In [18]:
def clean_rows(all_rows):
    new_data = []
    for row in all_rows:
        new_data.append(clean_text(row))
    return new_data

In [22]:
def load_imf_data():
    rdr = reader(open('../../data/imf_indicators.tsv', 'rb'), delimiter='\t')
    all_rows = [r for r in rdr if len(r) > 1]
    titles = all_rows.pop(0)
    cleaned_rows = clean_rows(all_rows)
    types = [text_type, text_type, text_type, text_type, text_type,
             number_type, number_type, number_type, number_type,
             number_type, number_type, number_type, number_type,
             date_type]
    return get_table(cleaned_rows, types, titles)

In [23]:
def add_last_percent_change():
    table = load_imf_data()
    table = table.where(lambda r: r.get('2015') is not
                        None and r.get('2014') is not None)
    table = table.where(lambda r: 'Unemployment' in
                        r.get('Subject Descriptor'))
    table = table.percent_change('2014', '2015', 'last_change')
    return table

In [24]:
table = add_last_percent_change()

In [25]:
table.columns['last_change'].mean()

Decimal('-2.595430263422718959419383707')

In [26]:
table.columns['2015'].mean()

Decimal('8.016756756756756756756756757')